<a href="https://colab.research.google.com/github/institutohumai/pyeph/blob/main/examples.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>


## *Instalación de la librería*

In [ ]:
!pip install pyeph

In [ ]:
import pyeph
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from  matplotlib.ticker import FuncFormatter

## *Obtención de las bases de datos*

In [ ]:
eph = pyeph.get(data="eph", year=2021, period=2, base_type='individual') 
canastas = pyeph.get(data="canastas") # canasta basica total y alimentaria
adequi = pyeph.get(data="adulto-equivalente") # adulto equivalente

In [ ]:
eph.head(10)

## *Estimación de Pobreza e Indigencia*

In [ ]:
pobreza = pyeph.Poverty(eph,canastas)

### *Pobreza Poblacional*

In [ ]:
# Tasa de pobreza de personas general 
pobreza.population()

In [ ]:
# Pobreza por sexo 
pobreza_sexo = pobreza.population(group_by='CH04')

pobreza_sexo

In [ ]:
pobreza_sexo_etiquetado = pyeph.etiquetar(pobreza_sexo)
pobreza_sexo_etiquetado

### *Pobreza Hogares*

In [ ]:
# Tasa de pobreza hogares 
pobreza.household()

## *Indicadores de mercado laboral*

In [ ]:
mercado_laboral = pyeph.LaborMarket(eph)

### *Desempleo*

In [ ]:
# Tasa de desempleo considerando la población total por sexo
desempleo_sexo = mercado_laboral.unemployment(group_by="CH04", div_by="PT")
desempleo_sexo


In [ ]:
desempleo_sexo_etiquetado = pyeph.etiquetar(desempleo_sexo)
desempleo_sexo_etiquetado

### *Empleo*

In [ ]:
# Tasa de desempleo considerando la Población en Edad de Trabajar
mercado_laboral.employment(div_by="PET")

In [ ]:
# Tasa de empleo considernado la Población Total por sexo
mercado_laboral.employment(group_by="CH04", div_by="PT")

In [ ]:
# Tasa de empleo general considerando la Población en Edad de Trabajar
mercado_laboral.employment(div_by="PET")

### *Actividad*

In [ ]:
# Tasa de actividad considerando la Población Total por sexo
mercado_laboral.activity(group_by="CH04", div_by="PT")

### *Vivienda*

Esta es una funcionalidad que podemos incorporar a la librería en el próximo taller.

¿Cuántos propietarios hay en los aglomerados urbanos? ¿Cómo evoluciona ese número?

In [ ]:
# Configuraciones generales de la visualización 
sns.set(rc = {'figure.figsize':(15,8)})

In [ ]:
desde = 2004
hasta = 2022

In [ ]:
regiones = {1:'Gran Buenos Aires',40:'NOA',41:'NEA',42:'Cuyo',43:'Pampeana',44:'Patagónica'}

In [ ]:
def get_propietarios_region(base, region):
    # Filtramos la región
    base = base.query(f'REGION=={region}')
    # Propietarios son códigos 1 y 2 a la pregunta II7
    prop = base.query('(II7==1) | (II7==2)')['PONDERA'].sum()
    total = base['PONDERA'].sum()
    return round((prop/ total) * 100,2)

In [ ]:
resultados = []
for anio in range(desde, hasta +1):
    try:
        base = pyeph.get(data="eph", year=anio, period=4, base_type='hogar') 
        for region_ix,region_desc  in regiones.items():
            propietarios_region = get_propietarios_region(base, region_ix)
            resultados.append({'region':region_desc, 'año': anio, 'propietarios_region':propietarios_region})
    except:
        pass

In [ ]:
resultados = pd.DataFrame(resultados)
resultados_wide = resultados.pivot("año", "region", "propietarios_region")

In [ ]:
sns.lineplot(data=resultados_wide)
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)));

In [ ]:
# 2 = Gran La Plata
# 3 = Bahía Blanca - Cerri
# 4 = Gran Rosario
# 5 = Gran Santa Fé
# 6 = Gran Paraná
# Encuesta Permanente de Hogares - INDEC 5
# 6 Encuesta Permanente de Hogares - INDEC
# 7 = Posadas
# 8 = Gran Resistencia
# 9 = Cdro. Rivadavia – Rada Tilly
# 10 = Gran Mendoza
# 12 = Corrientes
# 13 = Gran Córdoba
# 14 = Concordia
# 15 = Formosa
# 17 = Neuquén – Plottier
# 18 = S.del Estero - La Banda
# 19 = Jujuy - Palpalá
# 20 = Río Gallegos
# 22 = Gran Catamarca
# 23 = Salta
# 25 = La Rioja
# 26 = San Luis - El Chorrillo
# 27 = Gran San Juan
# 29 = Gran Tucumán - T. Viejo
# 30 = Santa Rosa - Toay
# 31 = Ushuaia - Río Grande
# 32 = Ciudad de Buenos Aires
# 33 = Partidos del GBA
# 34 = Mar del Plata - Batán
# 36 = Río Cuarto
# 38 = San Nicolás – Villa Constitución
# 91 = Rawson – Trelew
# 93 = Viedma – Carmen de Patagones
aglomerados = {5:'Gran Santa Fé',32:'Ciudad de Buenos Aires'}

In [ ]:
def get_propietarios_aglomerado(base, aglomerado):
    # Filtramos la región
    base = base.query(f'AGLOMERADO=={aglomerado}')
    # Propietarios son códigos 1 y 2
    prop = base.query('(II7==1) | (II7==2)')['PONDERA'].sum()
    total = base['PONDERA'].sum()
    return round((prop/ total) * 100,2)

In [ ]:
resultados = []
for anio in range(desde, hasta +1):
    try:
        base = pyeph.get(data="eph", year=anio, period=4, base_type='hogar') 
        for aglomerado_ix, aglomerado_desc  in aglomerados.items():
            propietarios_ag = get_propietarios_aglomerado(base, aglomerado_ix)
            resultados.append({'aglomerado':aglomerado_desc, 'año': anio, 'propietarios_ag':propietarios_ag})
    except:
        pass

In [ ]:
resultados = pd.DataFrame(resultados)
resultados_wide = resultados.pivot("año", "aglomerado", "propietarios_ag")

In [ ]:
sns.lineplot(data=resultados_wide)
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)));